<a href="https://colab.research.google.com/github/surabhiramamoorthy/Tomato_disease_detection/blob/main/New_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:










from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install anvil-uplink


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.7 MB/s eta 0:00:00


In [ ]:
import anvil.server

anvil.server.connect("G3VCHTEEYEN7R7IWZVGD5GMD-P3VP65NINOJ4WZ6S")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
# import necessary libraries (Keras)
# Keras supports all known type of layers: input, dense, convolutional, transposed convolution, reshape, normalization, dropout, flatten, and activation.
# Each layer performs a particular operation on the data.

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten

# To build a model we must now use the Model class
# The first two arguments it accepts represent the input and output layers.
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import *
from tensorflow.keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt

In [ ]:
!nvidia-smi

Sun Jun 30 10:11:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0              29W /  70W |    103MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# re-size all the images to this
IMAGE_SIZE=[224,224]
train_path='/content/drive/MyDrive/New_dataset_folder/train3'
valid_path='/content/drive/MyDrive/New_dataset_folder/test3'

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
# don't train existing weights in base model
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
#the glob module is used to retrieve files/pathnames matching a specified pattern.
#The pattern rules of glob follow standard Unix path expansion rules.
#It is also predicted that according to benchmarks it is faster than other methods to match pathnames in directories.
#With glob, we can also use wildcards ("*, ?, [ranges]) apart from
#exact string search to make path retrieval more simple and convenient.
folders=glob('/content/drive/MyDrive/New_dataset_folder/train3/*')

In [ ]:
folders

[]

In [ ]:
# our layers - more can be added if we want
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# create a model object
#To build a model we must now use the Model class
# The first two arguments it accepts represent the input and output layers.
model = Model(inputs=inception.input, outputs=prediction)


In [ ]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

In [ ]:
# tell the model what cost and optimization method to use
#-categorical crossentropy- Computes the cross-entropy loss between true labels and predicted labels.
# Adam optimization algorithm is an extension to stochastic gradient descent,
#-to update network weights iterative based on training data.
# A metric is a function that is used to judge the performance of your model.
#-Metric functions are similar to loss functions, except that the results from evaluating a metric are not used when
#-training the model. Accuracy class calculates how often predictions equal labels
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])
  model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
  )
  return model

In [ ]:
# Image Data Generator imports the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 0 images belonging to 1 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/New_dataset_folder/test3',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/New_dataset_folder/test3'

In [ ]:
import os
import tensorflow as tf
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
  )

In [ ]:
#Fit the model

r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=[cp_callback]


)

NameError: name 'test_set' is not defined

In [ ]:
model.save('model_new_dataset.h5')

In [ ]:
from tensorflow import keras
import tensorflow.keras.utils
from tensorflow.keras import utils as np_utils
model=keras.models.load_model('model_new_dataset.h5')


OSError: No file or directory found at model_new_dataset.h5

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
reloaded = tf.keras.models.load_model(
  '/content/model_new_dataset.h5',
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: name 'r' is not defined

In [ ]:
from tensorflow.keras.preprocessing import image


In [ ]:

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
other_datagen = ImageDataGenerator(rescale = 1./255)
other_set = other_datagen.flow_from_directory('/content/drive/MyDrive/Dataset/other_images',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2 images belonging to 1 classes.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
import anvil.server
import anvil.media
anvil.server.connect("G3VCHTEEYEN7R7IWZVGD5GMD-P3VP65NINOJ4WZ6S")

@anvil.server.callable
def model_predict(file):
    if isinstance(file,str):
      img = image.load_img(file, target_size=(224, 224))

      # Preprocessing the image
      x = image.img_to_array(img)
      # x = np.true_divide(x, 255)
      ## Scaling
      x=x/255
      x = np.expand_dims(x, axis=0)








      preds = model.predict(x)


      preds=np.argmax(preds, axis=1)
      if preds==0:
        preds="Bacterial_Spot"
      elif preds==1:
        preds="Early_blight"
      elif preds==2:
        preds="Late_blight"
      elif preds==3:
        preds="Leaf_Mold"
      elif preds==4:
        preds="Septoria_leaf_spot"
      elif preds==5:
        preds="Spider_mites Two-spotted_spider_mite"
      elif preds==6:
        preds="Target_Spot"
      elif preds==7:
        preds="Tomato_Yellow_Leaf_Curl_Virus"
      elif preds==8:
        preds="Tomato_mosaic_virus"
      else:
        preds="Healthy"



      return ("Prediction: {}".format())

    else:
      with anvil.media.TempFile(file) as f:

        img = image.load_img(f, target_size=(224, 224))

        # Preprocessing the image
        x = image.img_to_array(img)
        # x = np.true_divide(x, 255)
        ## Scaling
        x=x/255
        x = np.expand_dims(x, axis=0)








        preds = model.predict(x)


        preds=np.argmax(preds, axis=1)
        if preds==0:
          preds="Bacterial_Spot"
        elif preds==1:
          preds="Early_blight"
        elif preds==2:
          preds="Late_blight"
        elif preds==3:
          preds="Leaf_Mold"
        elif preds==4:
          preds="Septoria_leaf_spot"
        elif preds==5:
          preds="Spider_mites Two-spotted_spider_mite"
        elif preds==6:
          preds="Target_Spot"
        elif preds==7:
          preds="Tomato_Yellow_Leaf_Curl_Virus"
        elif preds==8:
          preds="Tomato_mosaic_virus"
        else:
          preds="Healthy"


        return ("Prediction: {}".format(preds))

anvil.server.wait_forever()


In [ ]:
plt.imshow(plt.imread('/content/drive/MyDrive/Dataset/train/Tomato___healthy/000bf685-b305-408b-91f4-37030f8e62db___GH_HL Leaf 308.1.JPG'))
model_predict('/content/drive/MyDrive/Dataset/train/Tomato___healthy/000bf685-b305-408b-91f4-37030f8e62db___GH_HL Leaf 308.1.JPG')

In [ ]:
while True:pass